In [6]:
from Bio import Entrez
import sqlite3
conn = sqlite3.connect('my.db')
c = conn.cursor()

c.execute("""CREATE TABLE genes 
                            (id INT, 
                            seqtype TEXT, 
                            name TEXT, 
                            description TEXT, 
                            chromosome TEXT, 
                            start INT, 
                            end INT, 
                            sequence TEXT); """)
c.execute("""CREATE TABLE pathways 
                            (name TEXT, 
                            description TEXT, 
                            enzymes TEXT); """)
c.execute("""CREATE TABLE enzymes 
                            (ec TEXT, 
                            name TEXT,
                            function TEXT) """)



OperationalError: table genes already exists

The method I created below is intended for an input 5 different strings and outputs a list of the name, sequence, ID number, and description of a desired entry in NCBI in a desired format. Respectively, (organism, dboi1, dboi2, ooi, format) refer to the organism of interest, the db argument desired for the Entrez.esearch function, the desired db argument for the Entrez.efetch function, the object of interest that we are looking up(e.g. 'hexokinase' if we are interested in an enzyme, or e.g. 'BRCA1' if we are interested in a gene) and the format argument for the Entrez.esearch, Entrez.efetch, and SeqIO.parse functions. 

In [ ]:
def dbentrez(organism, dboi1, dboi2, ooi, format):
    from Bio import SeqIO
    from Bio import Entrez
    null_output = 'nothing happened'
    Entrez.email = 'hurtadoj@berkeley.edu'
    pphandle = Entrez.esearch(db=dboi1,
                            term= [organism + ' [ORGN] '+ ooi] ,
                            sort='relevance',
                            idtype='acc')
    for i in Entrez.read(pphandle)['IdList']:
        prohandle = Entrez.efetch(db=dboi2, id=i, rettype = format, retmode = 'text')
        print('inner sleep 1')
        time.sleep(2.0)
    
        raw_thing = prohandle
        processed_thing = SeqIO.parse(raw_thing, format)
        for i in processed_thing:
            description = i.description
            if 'partial' in description:
                print("this one is partial")
            else:
                print('Full sequence of ' + str(ooi) +' available')
                if 'mRNA' in description:
                    print('this is mRNA')
                name = i.name
                seqp = i.seq
                seq = str(seqp)
                id = i.id
                desc = i.description
                output_list = [name, seq, id, desc]
                print(output_list)
                return(output_list)
    return(null_output)

*From top to bottom description of the bentrez method defined above*
Imports the necessary modules and instantiates the necessary variables. 
pphandle is created by using the input arguments in Entrez.esearch to scrape relevance-sorted accession numbers for the object of interest "ooi" in organism "organism" from the database dboi1. 

In [ ]:
def dbentrez(organism, dboi1, dboi2, ooi, format):
    from Bio import SeqIO
    from Bio import Entrez
    null_output = 'nothing happened'
    Entrez.email = 'hurtadoj@berkeley.edu'
    pphandle = Entrez.esearch(db=dboi1,
                            term= [organism + ' [ORGN] '+ ooi] ,
                            sort='relevance',
                            idtype='acc')

For every accession number in pphandle, a new handle is created with efetch, which retrieves data in the format 'format' from database 'dboi2' (a user of this dbentrez may want to esearch and efetch in 2 different databases). The function then prints a sleep message and sleeps for 2 seconds to avoid overloading the NCBI servers. 

In [ ]:
for i in Entrez.read(pphandle)['IdList']:
        prohandle = Entrez.efetch(db=dboi2, id=i, rettype = format, retmode = 'text')
        print('inner sleep 1')
        time.sleep(2.0)
    

The information in prohandle is then transferred to a new variable raw_thing. The information in raw_thing is parsed by SeqIO in its format, specified by the argument 'format'. We can now iterate through the "processed_thing" generator and extract valuable attributes (e.g. sequence, name, ID, description) using Bio.SeqIO as follows

In [ ]:
     raw_thing = prohandle
     processed_thing = SeqIO.parse(raw_thing, format)

The following code iterates through the generator output by SeqIO and sets a new variable named description to i's description attribute. Because I noticed that the most relevant efetch results often only contained partial sequences, I wanted to screen against this with the simple if statement below. If the string 'partial' is contained within any part of 'i's description, then the data in the database entry is not recorded and the user is informed with a print statement that the database entry likley contained only partial information. If "partial" is not in the entry's description, then a print message informs the user that a complete sequence has likely been found. If the sequence is an mRNA sequence, then 'mRNA' will likely be in the description (as I verified manually) and the user is informed of this too. A series of variables are then taken from i's attributes, which are ordered in a list. The list is printed and returned by the function. if for some reason 'processed_thing' is not iteratable because of some error, the null_output is returned, which is simply a string saying 'nothing happened' defined at the beginning of the method. 

In [ ]:
for i in processed_thing:
            description = i.description
            if 'partial' in description:
                print("this one is partial")
            else:
                print('Full sequence of ' + str(ooi) +' available')
                if 'mRNA' in description:
                    print('this is mRNA')
                name = i.name
                seqp = i.seq
                seq = str(seqp)
                id = i.id
                desc = i.description
                output_list = [name, seq, id, desc]
                print(output_list)
                return(output_list)
return(null_output)

_The code below uses dbentrez to generate the lists I need to iterate through to eventually populate my database_

In [ ]:
import time
list_of_organisms = ['homo sapiens', 'drosophila', 'escherichia coli']
list_of_pathways = ['glycolysis', 'TCA', 'PPP']
list_of_enzymes = [['hexokinase', 'glucokinase', 'phosphofructokinase','pyruvate kinase'],['citrate synthase', 'aconitase', 'isocitrate dehydrogenase','succinate dehydrogenase'],
                   ['glucose-6-phosphate dehydrogenase', 'gluconolactonase', '6-phosphogluconate dehydrogenase', 'Transaldolase']]



dict_of_pathways_and_their_enzymes = {}
for pathway in range(0,len(list_of_pathways)):
    dict_of_pathways_and_their_enzymes[list_of_pathways[pathway]]= list_of_enzymes[pathway]
dict_of_organisms_and_their_pathways = {}

for organism in list_of_organisms:
    for pathway in list_of_pathways:
        dict_of_organisms_and_their_pathways[organism] = dict_of_pathways_and_their_enzymes
empty_copy_for_output = {}

for organism in list_of_organisms:
    empty_copy_for_output[organism] = {}
    for empty_pathway in list_of_pathways:
        empty_copy_for_output[organism][empty_pathway] = []

## Using Entrez ##
import json
olist = dict_of_organisms_and_their_pathways.keys()
plist = dict_of_pathways_and_their_enzymes.keys()
for o in olist:
    print(o)
    # if o == 'escherichia coli' or o == 'homo sapiens':
        # continue
    for p in plist:
        print(p)
        temp_list_of_enzymes = []
        for enzyme in dict_of_organisms_and_their_pathways[o][p]:
            print(enzyme)
            useable_output = dbentrez(o,'nucleotide','nucleotide',enzyme, 'fasta')
            # print('USEABLE OUTPUT IS ')
            # print(useable_output)
            # print(useable_output)
            print('sleeping')
            time.sleep(2.0)
            # print('BEFORE: ' + str(empty_copy_for_output))
            empty_copy_for_output[o][p] += [useable_output]
            # print('AFTER: ' + str(empty_copy_for_output))
    print(empty_copy_for_output)
print(empty_copy_for_output)
empty_copy_for_output_string = str(empty_copy_for_output)
text = open('gdb.txt', 'w')
text.write(empty_copy_for_output_string)
text.close()
with open('gdb.json', 'w') as fp:
    json.dump(empty_copy_for_output, fp)

The code below imports the time module to allow me to call the sleep function. I also set lists that I will need to iterate through later containing the organisms, pathways, and enzymes of interest. Notably, the list_of_enzymes enzymes are separated into different lists depending on which pathway they were predetermined to belong to. 

In [ ]:
import time
list_of_organisms = ['homo sapiens', 'drosophila', 'escherichia coli']
list_of_pathways = ['glycolysis', 'TCA', 'PPP']
list_of_enzymes = [['hexokinase', 'glucokinase', 'phosphofructokinase','pyruvate kinase'],['citrate synthase', 'aconitase', 'isocitrate dehydrogenase','succinate dehydrogenase'],
                   ['glucose-6-phosphate dehydrogenase', 'gluconolactonase', '6-phosphogluconate dehydrogenase', 'Transaldolase']]

Defines an empty dictionary to contain pathways as keys to a list of their enzymes. This dictionary is populated by iterating through the previously defined list_of_pathways list. 

Next: defines an empty dictionary (dict_of_organisms_and_their_pathways) to become a nested dictionary containing organisms as keys to another dictoinary containing pathways as keys to a list of their enzymes. This dictionary will be used to iterate through the necessary set of arguments I need to put into the previously defined dbentrez method. 

Finally, an empty dictionary (empty_copy_for_output) list is define to be populated with the same keys as the previously defined nested dictionary. However, the pathway keys remain assigned to empty lists that will be populated later by the output of the previously defined dbentrez method. 

In [ ]:
dict_of_pathways_and_their_enzymes = {}
for pathway in range(0,len(list_of_pathways)):
    dict_of_pathways_and_their_enzymes[list_of_pathways[pathway]]= list_of_enzymes[pathway]
dict_of_organisms_and_their_pathways = {}

for organism in list_of_organisms:
    for pathway in list_of_pathways:
        dict_of_organisms_and_their_pathways[organism] = dict_of_pathways_and_their_enzymes
empty_copy_for_output = {}

for organism in list_of_organisms:
    empty_copy_for_output[organism] = {}
    for empty_pathway in list_of_pathways:
        empty_copy_for_output[organism][empty_pathway] = []

First line imports json
Second and third lines generate an iteratable variable (not technically a list) containing the organisms (olist) and pahtways(plist) to iterate through and input as arguments to dbentrez. 

Next, the code iterates through each organism and pathway. For each pathway it iterates through, it calls the dbentrez method previously defined, in this case inputting 'nucleotide' and 'nucleotide' for the dboi1 and dboi2 arguments.  Several print statements were included for debugging purposes and commented out. Then, a sleeping message is printed and the sleep function is called for 2 seconds to prevent overloading the NCBI servers. 

A given enzyme's output from dbentrez is then stored under the previously-defined empty_copy_for_output list under its respective organism and pathway. 

Finally, more print statements verify to the user that the code is outputting expected results and the output dictionary is saved as a text file and also as a json file to be loaded into another script below. 

In [ ]:
## Using Entrez ##
import json
olist = dict_of_organisms_and_their_pathways.keys()
plist = dict_of_pathways_and_their_enzymes.keys()
for o in olist:
    print(o)
    # if o == 'escherichia coli' or o == 'homo sapiens':
        # continue
    for p in plist:
        print(p)
        for enzyme in dict_of_organisms_and_their_pathways[o][p]:
            print(enzyme)
            useable_output = dbentrez(o,'nucleotide','nucleotide',enzyme, 'fasta')
            # print('USEABLE OUTPUT IS ')
            # print(useable_output)
            # print(useable_output)
            print('sleeping')
            time.sleep(2.0)
            # print('BEFORE: ' + str(empty_copy_for_output))
            empty_copy_for_output[o][p] += [useable_output]
            # print('AFTER: ' + str(empty_copy_for_output))
    print(empty_copy_for_output)
print(empty_copy_for_output)
empty_copy_for_output_string = str(empty_copy_for_output)
text = open('gdb.txt', 'w')
text.write(empty_copy_for_output_string)
text.close()
with open('gdb.json', 'w') as fp:
    json.dump(empty_copy_for_output, fp)

Imports json and uses it to import the outputs of dbentrez containing (from top to bottom) the dictionary containing lists with data for all enzymes in drosophila, ecoli, and humans. The fourth line opens a dictionary containing the relevant data for the defined genes in the define enzymes in all three defined organisms. All dictionaries were generated using the script above, including the previously-defined dbentrez method. The first three dictionaries were generated separately instead of together due to timeout errors from NCBI that I keept encountering whenever the script ran longer than roughly 5 minutes and the database of interest was 'enzymes'. 

In [ ]:
import json
with open('ProDict2drosoc.json', 'r') as fp:
    droso_dict = json.load(fp)
with open('ProDict2ecolic.json', 'r') as fp:
    ecoli_dict = json.load(fp)
with open('ProDict2humanc.json', 'r') as fp:
    human_dict = json.load(fp)
with open('gdb.json', 'r') as fp:
    genes_dict = json.load(fp)

The below code simply manually populates a bunch of lists with manually gathered data that I will need to iterate through to populate my sql tables. The functions in both 'list_of_enzyme_functions" and "list_of_pathway_functions" were gathered manually by copying and pasting information from each enzyme amd pathway's wikipedia page. 

In [ ]:
list_of_pathways = ['glycolysis', 'TCA', 'PPP']
list_of_enzymes = [['hexokinase', 'glucokinase', 'phosphofructokinase','pyruvate kinase'],['citrate synthase', 'aconitase', 'isocitrate dehydrogenase','succinate dehydrogenase'],
                   ['glucose-6-phosphate dehydrogenase', 'gluconolactonase', '6-phosphogluconate dehydrogenase', 'transaldolase']]
list_of_enzyme_functions = [['Phosphorylates hexoses',
                      'Facilitates phosphorylation of glucose to glucose-6-phosphate',
                      'Converts fructose-6-phosphate to fuctose-1,6-bisphosphate',
                      'Catalyzes transfer of phosphate group from PEP to ADP, yielding one pyruvate and one ATP'],
                     ['Catalyzes first reaction of TCA cycle by condensing acetyl-CoA and oxaloacetate to form citrate',
                       'Catalyzes the stereo-specific isomerization of citrate to isocitrate via cis-aconitate',
                      'catalyzes the oxidative decarboxylation of isocitrate, producing alpha-ketoglutarate (α-ketoglutarate) and CO2.',
                      'catalyzes the oxidation of succinate to fumarate with the reduction of ubiquinone to ubiquinol.'],
                     ['Catalyzes reaction of D-glucose 6-phosphate and NADP to 6-phospho-D-glucono-1,5-lactone + NADPH',
                      'catalyzes the chemical reaction D-glucono-1,5-lactone + H2O D-gluconate.',
                      'It is an oxidative carboxylase that catalyses the decarboxylating reduction of 6-phosphogluconate into ribulose 5-phosphate in the presence of NADP.',
                      'Catalyzes sedoheptulose 7-phosphate + glyceraldehyde 3-phosphate erythrose 4-phosphate + fructose 6-phosphate']
                     ]
list_of_pathway_functions = ['Breakdown of glucose by enzymes, releasing energy and pyruvic acid',
                             'Performs oxidation of acetyl-CoA into ATP and CO2 and produces the essential reducing agent NADH',
                             'Generates NADPH, pentoses, and ribose 5-phosphate, a crucial precursor for synthesizing nucleotides']
list_of_dicts = [human_dict, droso_dict, ecoli_dict]

The long for loop below simply generates a list for populating the 'enzyme' table (to be defined later) with the dictionaries generated by the dbentrez method and the script outline above. 

In [ ]:
master_list_of_enzyme_entries = []
for dict in list_of_dicts:
    keysindict = list((dict.keys()))
    relevant_index = list_of_dicts.index(dict)
    relevant_organism = keysindict[relevant_index]

    working_dict = dict[relevant_organism]
    local_pathways = list(working_dict.keys())
    for pathway in local_pathways:
        working_pathway = working_dict[pathway]
        pathway_index = local_pathways.index(pathway)
        for enzyme in working_pathway:
            enzyme_index = working_pathway.index(enzyme)
            new_enzyme_name = list_of_enzymes[pathway_index][enzyme_index]
            new_enzyme_ID = working_dict[pathway][enzyme_index][0]
            new_enzyme_aaseq = working_dict[pathway][enzyme_index][1]
            new_enzyme_organism = relevant_organism
            new_enzyme_pathway = pathway
            new_function = list_of_enzyme_functions[pathway_index][enzyme_index]
            print(new_enzyme_name)
            print(new_enzyme_ID)
            print(new_enzyme_aaseq)
            print(new_enzyme_organism)
            print(new_enzyme_pathway)
            print(new_function)

            new_entry = [new_enzyme_name,
                         new_enzyme_ID,
                         new_enzyme_aaseq,
                         new_enzyme_organism,
                         new_enzyme_pathway,
                         new_function]
            master_list_of_enzyme_entries.append(new_entry)

print('FINAL OUTPUT' + str(master_list_of_enzyme_entries))
#####

Does the same as above but for the lists to populate the pathways sql table. However, the lists iterated through to generate the final set of lists to populate the pathway table were all made from manually-organized information. 

In [ ]:
master_list_pathway_entries = []
for pathway in list_of_pathways:
    pathway_name = pathway
    pathway_description = list_of_pathway_functions[list_of_pathways.index(pathway)]
    pathway_enzymes = str(list_of_enzymes[list_of_pathways.index(pathway)])
    new_entry = [pathway_name, pathway_description, pathway_enzymes]
    master_list_pathway_entries.append(new_entry)

Does the same as above but for the lists to populate the gene sql table. The dictionaries iterated through ot make the necessary lists here were generated with the script and dbentrez method outline above. 

In [ ]:
master_list_gene_entries = []
all_organisms = list(genes_dict.keys())
for o in all_organisms:
    organism_pathways = list(genes_dict[o].keys())
    for p in organism_pathways:
        pathway_genes_list = genes_dict[o][p]
        for entry in pathway_genes_list:
            gene_name = entry[0]
            gene_seq = entry[1]
            gene_id = entry[2]
            gene_description = entry[3]
            gene_organism = o
            new_entry = [gene_id, gene_name, gene_description,gene_organism,gene_seq]
            master_list_gene_entries.append(new_entry)

Imports sqlite and creates the tables that I will populate in the following lines of code. 

In [ ]:
import sqlite3
selected_glycolysis_enzymes = ['hexokinase, glucokinase,'
                               'phosphofructokinase',
                               'pyruvate kinase']
selected_tcacycle_enzymes = ['citrate synthase', 'aconitase',
                             'isocitrate dehydrogenase',
                             'succinate dehydrogenase'
                             ]
selected_pentosephosphate_enzymes = ['glucose-6-phosphate dehydrogenase',
                                     'gluconolactonase',
                                     '6-phosphogluconate dehydrogenase',
                                     'ribulose-5-phosphate']

conn = sqlite3.connect('JHLab4.db')
c = conn.cursor()

c.execute("""CREATE TABLE genes 
                            (id TEXT,  
                            name TEXT, 
                            description TEXT,
                            organism TEXT,  
                            sequence TEXT); """)
c.execute("""CREATE TABLE pathways 
                            (name TEXT, 
                            description TEXT, 
                            enzymes TEXT); """)
c.execute("""CREATE TABLE enzymes 
                            (ec TEXT,
                            name TEXT,
                            aaseq TEXT, 
                            organism TEXT,
                            epathway TEXT,
                            efunction TEXT) """)


Adds the relevant information into each cell of each table from the "master_list_x_entries" lists, where "x" can be replaced with "enzyme", "pathway" or "gene to create a name of one of the lists I'm referring to. The conn.commit() command then commits the changes that I've made to the database. 

In [ ]:
for enzymeentry in master_list_of_enzyme_entries:
    c.execute(""" INSERT INTO enzymes(name, ec, aaseq, organism, epathway, efunction)"""
    """VALUES (?,?,?,?,?,?)""" , (enzymeentry[0],enzymeentry[1],
                                  enzymeentry[2],enzymeentry[3]
                                  ,enzymeentry[4],enzymeentry[5]))
for pathwayentry in master_list_pathway_entries:
    c.execute(""" INSERT INTO pathways(name, description, enzymes)""" 
              """VALUES(?,?,?)""", (pathwayentry[0], pathwayentry[1], pathwayentry[2]))
for pathwayentry in master_list_gene_entries:
    c.execute(""" INSERT INTO genes(id, name, description, organism, sequence)""" 
              """VALUES(?,?,?,?,?)""", (pathwayentry[0], pathwayentry[1], pathwayentry[2],
                                    pathwayentry[3],pathwayentry[4]))
conn.commit()